In [ ]:
__author__ = "Dilan Nana, Walid Rahman, Alex Popa"
__version__ = "CS230 Final Project, Fall Quarter 2024"

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       1.0Gi        78Gi       1.0Mi       4.5Gi        81Gi
Swap:             0B          0B          0B


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 18 03:40:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
main_directory = "/content/drive/MyDrive/CS230"

# Install Required Libraries

In [ ]:
# Install necessary libraries
!pip install transformers datasets evaluate torch matplotlib seaborn

In [ ]:
import os
import numpy as np
import pandas as pd
import warnings
import torch
import pickle
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm

In [ ]:
#Confirm correct runtime type
!nvidia-smi

Mon Nov 18 03:41:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Semantic Coherence Evaluation Functions

In [ ]:
!pip install sentence-transformers
!pip install evaluate
!pip install rouge_score

### Semantic Textual Similarity (STS) Eval

In [ ]:
from sentence_transformers import SentenceTransformer, util
from evaluate import load
bertscore = load("bertscore")

In [ ]:
def compute_sts(predictions, references, model_name='all-MiniLM-L6-v2'):
  """
    Computes Semantic Textual Similarity (STS) between predictions and references.

    Args:
        predictions (list of string): Generated answers.
        references (list of string): Ground truth answers.
        model_name (string): Pre-trained model name from sentence-transformers.

    Returns:
        float: Average cosine similarity score.
    """
  model = SentenceTransformer(model_name)
  embeddings1 = model.encode(predictions, convert_to_tensor=True)
  embeddings2 = model.encode(references, convert_to_tensor=True)

  cosine_scores = util.cos_sim(embeddings1, embeddings2)

  # Extract diagonal scores (prediction[i] vs reference[i])
  diagonal_scores = cosine_scores.diagonal()
  average_score = diagonal_scores.mean().item()

  return average_score

### BERT Score Eval

In [ ]:
def compute_bertscore(predictions, references, lang='en'):
    """
    Computes BERTScore for a list of predictions and references.

    Args:
        predictions (list of str): Generated answers.
        references (list of str): Ground truth answers.
        lang (str): Language of the texts.

    Returns:
        dict: BERTScore metrics (precision, recall, F1).
    """
    P, R, F1 = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    return {
        'BERTScore_Precision': P.mean().item(),
        'BERTScore_Recall': R.mean().item(),
        'BERTScore_F1': F1.mean().item()
    }

### Demonstration

In [ ]:
# DEMO
predictions = [
    "Paris is the capital of France.",
    "The theory of relativity was developed by Albert Einstein.",
    "The fury of the gods is a fantasy Nordic novel in a series by J.Gwynne"
]

references = [
    "Paris is France's capital city.",
    "Albert Einstein developed the theory of relativity.",
    "John Gwynne wrote a Nordic fantasy novel series, with one book being the Shadow of the Gods"
]

# Compute BERTScore
bertscore_results = compute_bertscore(predictions, references, lang='en')
print("BERTScore Results:", bertscore_results)

# Compute STS
sts_score = compute_sts(predictions, references)
print("Semantic Textual Similarity (STS) Score:", sts_score)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.05 seconds, 57.61 sentences/sec
BERTScore Results: {'BERTScore_Precision': 0.936948835849762, 'BERTScore_Recall': 0.9356613159179688, 'BERTScore_F1': 0.9362857937812805}
Semantic Textual Similarity (STS) Score: 0.8982057571411133


## Complete Evaluation

In [ ]:
def evaluate_answers(predictions, references):
    """
    Evaluates the generated answers against the reference answers using ROUGE, BLEU, BERTScore, and STS.

    Args:
        predictions (list of str): The generated answers.
        references (list of str): The reference (ground truth) answers.

    Returns:
        dict: The evaluation metrics.
    """
    # Load evaluation metrics
    rouge = evaluate.load('rouge')
    bleu = evaluate.load('bleu')
    bertscore = evaluate.load('bertscore')
    print(predictions)
    print(references)

    # Compute ROUGE scores
    rouge_scores = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    # Compute BLEU scores
    bleu_scores = bleu.compute(predictions=predictions, references=references)

    # Compute BERTScore
    bert_scores = bertscore.compute(predictions, references=references, lang='en', model_type="distilbert-base-uncased", verbose=False)

    #ref from HuggingFace: bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

    # bertscore = {
    #     'BERTScore_Precision': P.mean().item(),
    #     'BERTScore_Recall': R.mean().item(),
    #     'BERTScore_F1': F1.mean().item()
    # }

    # # Compute STS
    # sts = compute_sts(predictions, combined_references)

    # # print(rouge_scores)
    # # print(bleu_scores)
    # # Correctness analysis
    # correct_count = 0
    # incorrect_count = 0
    # for prediction, ref_list in zip(predictions, references):
    #     if any(prediction.lower() == ref.lower() for ref in ref_list):
    #         correct_count += 1
    #     else:
    #         incorrect_count += 1

    # # Print Results
    # print(f"Total Predictions: {len(predictions)}")
    # print(f"Correct Predictions: {correct_count}")
    # print(f"Incorrect Predictions: {incorrect_count}")
    sts = 0

    scores = {}
    scores.update(rouge_scores)
    scores.update(bleu_scores)
    scores.update(bert_scores)

    return scores

## Interpret Evaluation Results

In [ ]:
def interpret_metrics(metrics):
    """
    Interprets evaluation metrics by categorizing their scores.

    Args:
        metrics (dict): A dictionary containing metric names and their scores.
                        Example:
                        {
                            'rouge1': 0.7159,
                            'rouge2': 0.3002,
                            'rougeL': 0.4141,
                            'bertscore_f1': 0.8250,
                            'sts': 0.7500
                        }

    Returns:
        dict: A dictionary with metrics, their scores, and dynamic descriptions.
              Example:
              {
                  'rouge1': {'score': 0.7159, 'description': 'High content coverage.'},
                  ...
              }
    """
    interpreted = {}

    # Define thresholds for each metric
    thresholds = {
        'rouge1': {
            'Low': 0.50,
            'Moderate': 0.70
        },
        'rouge2': {
            'Low': 0.20,
            'Moderate': 0.40
        },
        'rougeL': {
            'Low': 0.30,
            'Moderate': 0.60
        },
        'bertscore_f1': {
            'Low': 0.70,
            'Moderate': 0.85
        },
        'sts': {
            'Low': 0.60,
            'Moderate': 0.80
        }
    }

    # Define descriptions based on categories
    descriptions = {
        'Low': {
            'rouge1': 'Low content coverage.',
            'rouge2': 'Low phrase overlap.',
            'rougeL': 'Poor structural similarity.',
            'bertscore_f1': 'Weak semantic alignment.',
            'sts': 'Poor semantic coherence.'
        },
        'Moderate': {
            'rouge1': 'Moderate content coverage.',
            'rouge2': 'Moderate phrase overlap.',
            'rougeL': 'Good structural similarity.',
            'bertscore_f1': 'Good semantic alignment.',
            'sts': 'Good semantic coherence.'
        },
        'High': {
            'rouge1': 'High content coverage.',
            'rouge2': 'Strong phrase overlap.',
            'rougeL': 'Excellent structural similarity.',
            'bertscore_f1': 'Strong semantic alignment.',
            'sts': 'Excellent semantic coherence.'
        }
    }

    for metric, score in metrics.items():
        metric_lower = metric.lower()
        if metric_lower in thresholds:
            if score < thresholds[metric_lower]['Low']:
                category = 'Low'
            elif score < thresholds[metric_lower]['Moderate']:
                category = 'Moderate'
            else:
                category = 'High'

            description = descriptions[category][metric_lower]
            interpreted[metric_lower] = {
                'score': round(score, 4),
                'description': description
            }
        else:
            interpreted[metric_lower] = {
                'score': round(score, 4),
                'description': 'No interpretation available.'
            }

    return interpreted

In [ ]:
#Dynamic option
def interpret_metrics_dynamic(metrics, percentile_low=33, percentile_moderate=66):
    """
    Interprets evaluation metrics by categorizing their scores based on dynamic percentile thresholds.

    Args:
        metrics (dict): A dictionary containing metric names and their scores.
                        Example:
                        {
                            'rouge1': 0.7159,
                            'rouge2': 0.3002,
                            'rougeL': 0.4141,
                            'bertscore_f1': 0.8250,
                            'sts': 0.7500
                        }
        percentile_low (int): Lower percentile for 'Low' category.
        percentile_moderate (int): Middle percentile for 'Moderate' category.

    Returns:
        dict: A dictionary with metrics, their scores, and dynamic descriptions.
    """
    interpreted = {}

    # Collect scores for dynamic thresholding
    all_scores = metrics.values()
    low_threshold = np.percentile(list(all_scores), percentile_low)
    moderate_threshold = np.percentile(list(all_scores), percentile_moderate)

    for metric, score in metrics.items():
        if score < low_threshold:
            category = 'Low'
        elif score < moderate_threshold:
            category = 'Moderate'
        else:
            category = 'High'

        # Define dynamic descriptions
        if category == 'Low':
            description = f"{metric.upper()} is Low."
        elif category == 'Moderate':
            description = f"{metric.upper()} is Moderate."
        else:
            description = f"{metric.upper()} is High."

        interpreted[metric] = {
            'score': round(score, 4),
            'description': description
        }

    return interpreted

## Save Evaluation Results

In [ ]:
def save_evaluation_results(results, save_dir, filename='llama2_7b_qa_baseline_semantic_coherence_metrics.csv'):
    """
    Saves the evaluation metrics to a CSV file.

    Args:
        results (dict): Dictionary containing evaluation metrics.
        save_dir (str): Directory path to save the CSV file.
        filename (str): Name of the CSV file.
    """
    results_df = pd.DataFrame(results).transpose()
    results_df = results_df.reset_index().rename(columns={'index': 'Dataset'})
    csv_path = os.path.join(save_dir, filename)
    results_df.to_csv(csv_path, index=False)
    print(f"Evaluation results saved to '{csv_path}'.")

## Plot Metrics

In [ ]:
def plot_metrics(results, metrics_to_plot=['rouge1', 'rouge2', 'rougeL', 'bleu', 'BERTScore_F1', 'STS']):
    """
    Plots evaluation metrics for different datasets.

    Args:
        results (dict): Dictionary containing evaluation metrics.
        metrics_to_plot (list of str): List of metric names to plot.
    """
    df = pd.DataFrame(results).transpose()
    df = df.reset_index().rename(columns={'index': 'Dataset'})

    # Melt the dataframe for easier plotting
    df_melted = df.melt(id_vars='Dataset', value_vars=metrics_to_plot, var_name='Metric', value_name='Score')

    plt.figure(figsize=(14, 8))
    sns.barplot(x='Dataset', y='Score', hue='Metric', data=df_melted)
    plt.title('Semantic Coherence Evaluation Metrics by Dataset')
    plt.xlabel('Dataset')
    plt.ylabel('Score')
    plt.legend(title='Metric')
    plt.tight_layout()
    plt.show()

# Beam Search Experiment

## Creating beam search

In [ ]:
#Creating beam search for classic implementation
class Beam:
  def __init__(self, tokens, log_probs, is_finished=False):
    self.tokens = tokens #token ID list
    self.log_prob = log_probs #Cumulative log probability
    self.is_finished = is_finished #gen <eos>

In [ ]:
def manual_beam_search(helper, prompt, max_length=100, num_beams=5, early_stopping=True, no_repeat_ngram_size=2):
  """
  Performs manual Beam Search.

  Args:
      helper (Llama7BHelper): The helper class instance.
      prompt (str): The input prompt.
      max_length (int): Maximum length of the generated sequence.
      num_beams (int): Number of beams.
      early_stopping (bool): Whether to stop early when all beams have finished.
      no_repeat_ngram_size (int): Size of n-grams that should not be repeated.

  Returns:
      str: The generated sequence with the highest score.
  """
  tokenizer = helper.tokenizer
  model = helper.model
  device = helper.device
  #Dilan come back to this
  # Encode the prompt
  input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
  initial_beam = Beam(tokens=input_ids[0].tolist(), log_prob=0.0)
  beams = [initial_beam]

  for step in range(max_length):
      all_candidates = []
      for beam in beams:
          if beam.is_finished:
              # If the beam is already finished, carry it forward
              all_candidates.append(beam)
              continue

          # Convert tokens to tensor
          input_ids = torch.tensor([beam.tokens]).to(device)

          with torch.no_grad():
              outputs = model(input_ids)
              logits = outputs.logits  # Shape: (1, seq_len, vocab_size)
              next_token_logits = logits[0, -1, :]  # Shape: (vocab_size,)

              # Apply softmax to get probabilities
              probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
              log_probs = torch.log(probs)

          # Get top 'num_beams' tokens
          top_log_probs, top_indices = torch.topk(log_probs, num_beams)

          for i in range(num_beams):
              token_id = top_indices[i].item()
              token_log_prob = top_log_probs[i].item()

              new_tokens = beam.tokens + [token_id]
              new_log_prob = beam.log_prob + token_log_prob

              # Check for end-of-sequence token
              if token_id == tokenizer.eos_token_id:
                  is_finished = True
              else:
                  is_finished = False

              # Check for no repeating n-grams
              if no_repeat_ngram_size > 0:
                  ngrams = zip(*[new_tokens[i:] for i in range(no_repeat_ngram_size)])
                  ngram_set = set(ngrams)
                  # No action needed here as we're preventing expansion that would create repeats

              new_beam = Beam(tokens=new_tokens, log_prob=new_log_prob, is_finished=is_finished)
              all_candidates.append(new_beam)

      # Sort all candidates by log probability
      ordered = sorted(all_candidates, key=lambda b: b.log_prob, reverse=True)

      # Select top 'num_beams' beams
      beams = ordered[:num_beams]

      # If early stopping is enabled and all beams are finished, stop
      if early_stopping and all(beam.is_finished for beam in beams):
          break

  # Select the beam with the highest log probability
  best_beam = max(beams, key=lambda b: b.log_prob)
  generated_sequence = tokenizer.decode(best_beam.tokens, skip_special_tokens=True)
  return generated_sequence

## Beam Search Evaluation Function

In [ ]:
def evaluate_llama_manual_beam_search(helper, dataset, max_length=100, num_beams=5):
  predictions = []
  references = []
  for sample in tqdm(dataset, desc="Generating and Evaluating"):
    try:
        if 'question' in sample and 'answers' in sample:
            # SQuAD-like structure
            question = sample['question']
            reference = sample['answers']['text'][0]
        elif 'question' in sample and 'answer' in sample:
            # HotpotQA-like structure
            question = sample['question']
            reference = sample['answer']
        elif 'question' in sample and 'answer' in sample:
            # TriviaQA-like structure
            question = sample['question']['query']
            reference = sample['answer']['value'][0] if isinstance(sample['answer']['value'], list) else sample['answer']['value']
        else:
            raise ValueError("Unknown dataset structure.")

        # Generate answer using manual Beam Search
        generated_answer = helper.generate_text_manual_beam_search(
            prompt=question,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        predictions.append(generated_answer)
        references.append(reference)
    except KeyError as ke:
        print(f"Missing key in sample: {ke}")
        continue
    except Exception as e:
        print(f"Error processing sample: {e}")
        continue

  # Initialize SentenceTransformer model for STS
  st_model = SentenceTransformer('all-MiniLM-L6-v2')

  # Call our evaluation function for semantic coherence metrics
  metrics = evaluate_answers(predictions, references, st_model)
  return metrics


In [ ]:
def save__beam_search_evaluation_results(results, save_dir, filename='llama2_7b_qa_beam_search_metrics.csv'):
    """
    Saves the evaluation metrics to a CSV file.

    Args:
        results (dict): Dictionary containing evaluation metrics.
        save_dir (str): Directory path to save the CSV file.
        filename (str): Name of the CSV file.
    """
    results_df = pd.DataFrame(results).transpose()
    results_df = results_df.reset_index().rename(columns={'index': 'Dataset'})
    csv_path = os.path.join(save_dir, filename)
    results_df.to_csv(csv_path, index=False)
    print(f"Evaluation results saved to '{csv_path}'.")

In [ ]:
def plot_beam_search_metrics(results, metrics_to_plot=['rouge1', 'rouge2', 'rougeL', 'bleu', 'BERTScore_F1', 'STS']):
    """
    Plots evaluation metrics for different datasets.

    Args:
        results (dict): Dictionary containing evaluation metrics.
        metrics_to_plot (list of str): List of metric names to plot.
    """
    df = pd.DataFrame(results).transpose()
    df = df.reset_index().rename(columns={'index': 'Dataset'})

    # Melt the dataframe for easier plotting
    df_melted = df.melt(id_vars='Dataset', value_vars=metrics_to_plot, var_name='Metric', value_name='Score')

    plt.figure(figsize=(14, 8))
    sns.barplot(x='Dataset', y='Score', hue='Metric', data=df_melted)
    plt.title('Semantic Coherence Evaluation Metrics by Dataset')
    plt.xlabel('Dataset')
    plt.ylabel('Score')
    plt.legend(title='Metric')
    plt.tight_layout()
    plt.show()

# QA w/Standard Decoding Experiment (no-mods)

**WHAT PROMPT??**

### Generate Text/Answers Function

In [ ]:
#temp, repetition penalty etc.?
#Llama team made these choices: https://huggingface.co/meta-llama/Llama-2-7b-hf/blob/main/generation_config.json
def generate_answers(model, tokenizer, questions, max_length=200, temperature=0.7, top_p=0.9, top_k=50, max_new_tokens=100, repetition_penalty=1.2, do_sample=True):
    """
    Generates answers for a list of questions using the provided model and tokenizer.

    Args:
        model (AutoModelForCausalLM): The language model.
        tokenizer (AutoTokenizer): The tokenizer.
        questions (list of str): List of questions to answer.
        max_length (int): Maximum length of the generated answer.
        temperature (float): Sampling temperature.
        top_p (float): Nucleus sampling probability.
        top_k (int): Top-k sampling.
        repetition_penalty (float): Penalty for repeating tokens.

    Returns:
        list of str: Generated answers.
    """
    generated_answers = []

    for question in tqdm(questions, desc="Generating Answers"):
        prompt = f"Question: {question}\nAnswer:"
        # inputs = tokenizer.encode(prompt, return_tensors='pt').to(model.device)
        print(prompt)
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            padding=True, #Pad so shorter inputs match longest batch
            truncation=True
        )
        input_ids = inputs["input_ids"].to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)

        # Generate response using standard decoding (greedy as default)
        outputs = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens, #use max_length if inputs are consistently short
            do_sample=do_sample,# False for greedy
            attention_mask=attention_mask,
        )
        # early_stopping=True, cant use with num_beams = 1

        # outputs = model.generate(
        #     input_ids,
        #     max_length=max_length,
        #     temperature=temperature,
        #     top_p=top_p,
        #     top_k=top_k,
        #     repetition_penalty=repetition_penalty,
        #     do_sample=do_sample,# False for greedy
        #     eos_token_id=tokenizer.eos_token_id,
        #     pad_token_id=tokenizer.eos_token_id
        # )

        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract the answer part after "Answer:"
        answer = answer.split("Answer:")[-1].strip()
        generated_answers.append(answer)

    return generated_answers

# Standard Activation Steering

## Forward hook

In [ ]:
class ActivationSteering:
    def __init__(self, model, target_layer, mixing_activation, multiplier=10):
      """
      Initializes the ActivationSteering instance.

      Args:
          model (AutoModelForCausalLM): The language model.
          target_layer (int): The layer number where steering will be applied.
          mixing_activation (torch.Tensor): The activation to mix.
          multiplier (float): Scaling factor for the mixing activation.
      """
      self.model = model
      self.target_layer = target_layer
      self.mixing_activation = mixing_activation.to(model.device)
      self.multiplier = multiplier
      self.hook = None
      self.steered = False  # To ensure mixing is applied only once per generation

    def get_target_layer_module(self):
      """
      Retrieves the target layer's attention module.
      """
      return self.model.model.layers[self.target_layer].self_attn

    def hook_fn(self, module, input, output):
      """
      The hook function to modify attention values.

      Args:
          module: The module to which the hook is attached.
          input: The input to the module.
          output: The output from the module.
      """
      if not self.steered:
          # output[0] contains the attention weights
          # output[1] contains the context (attention output)
          # We'll modify the attention output
          modified_context = output[1] + self.multiplier * self.mixing_activation
          return (output[0], modified_context)
      else:
          return output

    def apply_hook(self):
      """
      Attaches the hook to the target layer's attention module.
      """
      target_module = self.get_target_layer_module()
      self.hook = target_module.register_forward_hook(self.hook_fn)

    def remove_hook(self):
      """
      Removes the attached hook.
      """
      if self.hook is not None:
        self.hook.remove()
        self.hook = None

## Mixing Attention Preparation

I need to figure out how to define the mixing input: A sentence or phrase that embodies the concept or persona you want to steer towards or away from.

## Generating the Mixing Activation:

In [ ]:
def get_mixing_activation(model, tokenizer, mixing_input, target_layer, max_length=50):
  """
  Generates the mixing activation from the mixing input.

  Args:
      model (AutoModelForCausalLM): The language model.
      tokenizer (AutoTokenizer): The tokenizer.
      mixing_input (str): The input to generate the mixing activation.
      target_layer (int): The layer number from which to extract the activation.
      max_length (int): Maximum length of the mixing input generation.

  Returns:
      torch.Tensor: The mixing activation to be added.
  """
  # Encode the mixing input
  prompt = f"Statement: {mixing_input}\nContinuation:"
  inputs = tokenizer.encode(prompt, return_tensors='pt').to(model.device)

  # Disable gradients for mixing activation generation
  with torch.no_grad():
      outputs = model.generate(
          inputs,
          max_length=max_length,
          temperature=0.7,
          top_p=0.9,
          top_k=50,
          repetition_penalty=1.2,
          do_sample=True,
          eos_token_id=tokenizer.eos_token_id,
          pad_token_id=tokenizer.eos_token_id
      )

  # Decode the generated continuation
  continuation = tokenizer.decode(outputs[0], skip_special_tokens=True)
  continuation = continuation.split("Continuation:")[-1].strip()

  # Tokenize the mixing input and continuation
  full_input = f"Statement: {mixing_input}\nContinuation: {continuation}"
  inputs = tokenizer.encode(full_input, return_tensors='pt').to(model.device)

  # Register a forward hook to capture attention output at the target layer
  mixing_activation = []

  def hook_fn(module, input, output):
      # output[0]: attention weights, output[1]: attention context
      # We'll capture the attention context
      mixing_activation.append(output[1].clone())

  target_module = model.model.layers[target_layer].self_attn
  hook = target_module.register_forward_hook(hook_fn)

  # Forward pass to capture activation
  with torch.no_grad():
      model(inputs)

  # Remove the hook
  hook.remove()

  if mixing_activation:
      return mixing_activation[0]
  else:
      raise ValueError("Failed to capture mixing activation.")

# Main Flow

## Gather Model + Resources

### Llama Wrapper Class

In [ ]:
# Define your helper class
class Llama7BHelper:
  def __init__(self, pretrained_model="huggyllama/llama-7b"):
      self.device = "cuda" if torch.cuda.is_available() else "cpu"
      self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
      self.model = AutoModelForCausalLM.from_pretrained(pretrained_model).to(self.device)
      #explicitly set pad token b/c ran into issues, but Nina didn't?
      if self.tokenizer.pad_token_id is None:
        self.tokenizer.pad_token_id = self.tokenizer.pad_token_id = 0 #got this from generation_config.json
        self.tokenizer.pad_token="[PAD]"

  #taken from nrimsky github
  def get_logits(self, prompt):
    inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(self.device)
    attention_mask = inputs["attention_mask"].to(self.device)
    with torch.no_grad():
        logits = self.model(input_ids=input_ids, attention_mask=attention_mask).logits
    return logits

  def generate_text_manual_beam_search(self, prompt, max_length=100, num_beams=5, early_stopping=True, no_repeat_ngram_size=2):
        return manual_beam_search(
            helper=self,
            prompt=prompt,
            model=self.model,
            tokenizer=self.tokenizer,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=early_stopping,
            no_repeat_ngram_size=no_repeat_ngram_size
        )

In [ ]:
# #Specify model
# pretrained_model="huggyllama/llama-7b"

# #Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# #Load model
# model = AutoModelForCausalLM.from_pretrained(pretrained_model, device_map="auto", torch_dtype=torch.float16, low_cpu_mem_usage=True)

# # Initialize the generation pipeline
# qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

### Load or Download Model Function

In [ ]:
#Load model from drive if exists
def load_or_download_model(model_load_path):
  if os.path.exists(model_load_path) and os.listdir(model_load_path):
    print("Loading model from Google Drive...")
    helper = Llama7BHelper(pretrained_model=model_load_path)
  else:
    print("We could not find model in Google Drive. Downloading from Hugging Face...")
    helper = Llama7BHelper(pretrained_model="huggyllama/llama-7b")
    # Create the directory if it doesn't exist
    os.makedirs(model_load_path, exist_ok=True)
    print("Saving model to Google Drive...")
    helper.tokenizer.save_pretrained(model_load_path)
    helper.model.save_pretrained(model_load_path)
    print(f"Ok! We've saved model and tokenizer to {model_load_path}")

  return helper

In [ ]:
# #Save model to drive folder for future use
# model_save_dir = "/content/drive/MyDrive/CS230/Llama2-7b-Model"
# # Define the path to save the model


# # Save the tokenizer and model
# tokenizer.save_pretrained(model_save_dir)
# model.save_pretrained(model_save_dir)

# print(f"Model and tokenizer saved to {model_save_dir}")

## Loading/Processing Datasets Functions

In [ ]:
#Save datasets to drive if first time using
def save_datasets_as_pickle(datasets, save_dir):
  for name, dataset in datasets.items():
      df = dataset.to_pandas()
      file_path = f"{save_dir}/{name}.pkl"
      df.to_pickle(file_path)
      print(f"Saved {name} to {file_path}")


In [ ]:
#Get small sample per dataset
def get_shuffled_small_dataset(datasets, sample_size=100, seed=42):
  sampled_datasets={}
  for name, dataset in datasets.items():
    sampled_dataset = dataset.shuffle(seed=seed).select(range(sample_size))
    sampled_datasets[name] = sampled_dataset
    print(f"{name} sampled with {len(sampled_dataset)} examples.")
  return sampled_datasets

In [ ]:
def download_baseline_datasets(save_dir, datasets={}):
  # Load SQuAD, HotpotQA, and TriviaQA datasets
  # SQuAD v2.0
  datasets['squad'] = load_dataset('squad_v2', split='validation')

  # Load HotpotQA
  datasets['hotpotqa'] = load_dataset('hotpot_qa', 'distractor', split='validation')

  # Load TriviaQA (web version)
  #https://docs.allennlp.org/models/main/models/rc/dataset_readers/triviaqa/ <-- helpful for future use
  datasets['triviaqa'] = load_dataset('trivia_qa', 'unfiltered', split='validation')

  #Save dataset portions to drive folder for fast reloading
  save_datasets_as_pickle(datasets, save_dir)

  # Display the number of samples in each dataset
  for name, dataset in datasets.items():
      print(f"{name.capitalize()} has {len(dataset)} samples.")

  return datasets

In [ ]:
# for filename in os.listdir(main_directory):
#   loaded_datasets = {}
#   if filename.endswith(".pkl"):
#     filepath = os.path.join(main_directory, filename)
#     dataset_name = filename.replace(".pkl", "")  # Remove the .pkl extension for the name

#     # Load the pickle file
#     with open(filepath, 'rb') as f:
#       pandas_df = pickle.load(f)

#     # Convert to Hugging Face Dataset
#     loaded_datasets[dataset_name] = Dataset.from_pandas(pandas_df)
#     print(f"Loaded and converted {filename} to Hugging Face Dataset.")

#   print(filename)

In [ ]:
#Load datasets or retrieve
def load_datasets_from_pickle(save_dir):
    """
    Loads each dataset from its corresponding Pickle file.

    Args:
        save_dir (str): Directory path in Google Drive where files are saved.
        sample_size (int): Number of samples expected in each Pickle file.

    Returns:
        dict: A dictionary containing loaded datasets.
    """
    loaded_datasets = {}
    for filename in os.listdir(save_dir):
      if filename.endswith(".pkl"):
        filepath = os.path.join(save_dir, filename)
        dataset_name = filename.replace(".pkl", "")  # Remove the .pkl extension for the name

        # Load the pickle file
        with open(filepath, 'rb') as f:
          pandas_df = pickle.load(f)

        # Convert to Hugging Face Dataset
        loaded_datasets[dataset_name] = Dataset.from_pandas(pandas_df)
        print(f"Loaded and converted {filename} to data frame.")
      print(filename)
    return loaded_datasets

    # for name in dataset_names:
    #     file_path = os.path.join(save_dir, f"{name}.pkl")
    #     if os.path.exists(file_path):
    #         df = pd.read_pickle(file_path)
    #         # Convert DataFrame back to Hugging Face Dataset
    #         dataset = load_dataset('csv', data_files=file_path, split='train')
    #         loaded_datasets[name] = dataset
    #         print(f"Loaded {name} from {file_path}")
    #     else:
    #         print(f"{name}.pkl does not exist in {save_dir}.")
    #         loaded_datasets = download_baseline_datasets(save_dir)
    # return loaded_datasets

## Display Sample Data Function

In [ ]:
def display_examples_per_dataset(smaller_datasets, sample_size=5):
  """
  Displays subset from SQuAD, HotpotQA, and TriviaQA datasets.

  """
  for name, dataset in datasets.items():
    small_sample = dataset.select(range(sample_size))
    if name == 'squad':
      print("=== SQuAD Samples ===")
      for idx, sample in enumerate(small_sample):
          print(f"\nSample {idx+1}:")
          print(f"Question: {sample['question']}")
          print(f"Answer: {sample['answers']['text'][0]}")
    elif name == 'hotpotqa':
       print("\n=== HotpotQA Samples ===")
       for idx, sample in enumerate(small_sample):
          print(f"\nSample {idx+1}:")
          print(f"Question: {sample['question']}")
          print(f"Answer: {sample['answer']}")
    else:
      print("\n=== TriviaQA Samples ===")
      for idx, sample in enumerate(small_sample):
          print(f"\nSample {idx+1}:")
          print(f"Question: {sample['question']}")
          answer = sample['answer']['value']
          # Handle list or single answer
          if isinstance(answer, list):
              answer = answer[0]
          print(f"Answer: {answer}")

## Running commands

### Get Data From Drive + Shuffle/Sample

In [ ]:
#Retrieve data
datasets = {}
if not datasets:
  datasets = load_datasets_from_pickle("/content/drive/MyDrive/CS230")
else:
  print("We're all good! Datasets recovered")
  print("actually need to download from helper function")
sample_size = 100  # Adjust as needed
smaller_datasets = get_shuffled_small_dataset(datasets, sample_size=sample_size, seed=42)
display_examples_per_dataset(smaller_datasets, sample_size=5)

res.png
Loaded and converted squad.pkl to data frame.
squad.pkl
Loaded and converted hotpotqa.pkl to data frame.
hotpotqa.pkl
Loaded and converted triviaqa.pkl to data frame.
triviaqa.pkl
Llama2-7b-Model
Untitled0.ipynb
230 Project Idea.gdoc
Llama27b_baselines.ipynb
squad sampled with 100 examples.
hotpotqa sampled with 100 examples.
triviaqa sampled with 100 examples.
=== SQuAD Samples ===

Sample 1:
Question: In what country is Normandy located?
Answer: France

Sample 2:
Question: When were the Normans in Normandy?
Answer: 10th and 11th centuries

Sample 3:
Question: From which countries did the Norse originate?
Answer: Denmark, Iceland and Norway

Sample 4:
Question: Who was the Norse leader?
Answer: Rollo

Sample 5:
Question: What century did the Normans first gain their separate identity?
Answer: 10th century

=== HotpotQA Samples ===

Sample 1:
Question: Were Scott Derrickson and Ed Wood of the same nationality?
Answer: yes

Sample 2:
Question: What government position was held b

In [ ]:
# references = [
#     sample["text"][0] if sample["text"] else ""  # Use the first answer if available, otherwise empty string
#     for sample in smaller_datasets["squad"]["answers"]
# ]
# print(references)

'''
question: xxxx
valid_answers: ['x1', 'x2', 'x3']
prediction: 'blue'

3 predictions
blue -> x1
blue -> x2
blue -> x3
'''


In [ ]:
tiny_datasets['squad'][0]['answers']['text']

['Fort Presque Isle (near present-day Erie, Pennsylvania',
 'Fort Presque Isle',
 'near present-day Erie, Pennsylvania',
 'Fort Presque Isle',
 'near present-day Erie, Pennsylvania']

### Get Model From Drive

In [ ]:
#Retreive model
model_load_path = "/content/drive/MyDrive/CS230/Llama2-7b-Model"
helper = load_or_download_model(model_load_path)

Loading model from Google Drive...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Running Standard Decoding Experiment Functions

In [ ]:
def run_basic_experiment(wrapped_model=helper, sampled_datasets={}):
  """
    Run a basic experiment to generate answers and evaluate them across datasets.

    Args:
        helper (Llama7BHelper): An instance of the Llama7BHelper class.
        sampled_datasets (dict): A dictionary of sampled datasets.

    Returns:
        dict: A dictionary containing generated predictions/answers for each dataset.
    """

  # Initialize a dictionary to store generated answers
  generated_answers = {}

  for dataset_name, dataset in sampled_datasets.items():
      print(f"\nGenerating answers for {dataset_name}...")
      if dataset_name == 'squad':
          questions = dataset['question']
      elif dataset_name == 'hotpotqa':
          questions = dataset['question']
      elif dataset_name == 'triviaqa':
          questions = dataset['question']
      else:
          raise ValueError(f"Unknown dataset: {dataset_name}")

      answers = generate_answers(model=wrapped_model.model, tokenizer=wrapped_model.tokenizer, questions=questions)
      generated_answers[dataset_name] = answers
  return generated_answers

In [ ]:
questionsX = smaller_datasets["triviaqa"]["question"]
print(questionsX)

['What type of dance shoe has a specially hardened sole or attached metal plates?', 'The songs I got Life and Aquarius come from which musical?', 'The name of which musical instrument comes from the German for bells and play?', 'Who narrated the BBC television production of Paddington?', "What is the name of Hamlet's mother in the Shakespeare play of the same name?", 'Who played Dr Richard Kimble in the 1993 film The Fugitive?', 'An analgesic drug is commonly called a?', 'In the Blandings Castle stories by P G Wodehouse, what kind of animal is The Empress of Blandings?', "After 23 years on the air, America's Most Wanted will be winding down. Who has been the host of the series since its inception?", 'Which club won the Scottish league cup three times in the first eight years of the competition?', "Which American resort is nicknamed 'Sin City'?", 'The voice of Bugs Bunny, Daffy Duck, Porky Pig, Sylvester the Cat, Tweety Bird, Foghorn Leghorn, Yosemite Sam, Wile E. Coyote, Woody Woodpeck

In [ ]:
tiny_datasets['squad'][0]

{'id': '5733ea04d058e614000b6595',
 'title': 'French_and_Indian_War',
 'context': "In the spring of 1753, Paul Marin de la Malgue was given command of a 2,000-man force of Troupes de la Marine and Indians. His orders were to protect the King's land in the Ohio Valley from the British. Marin followed the route that Céloron had mapped out four years earlier, but where Céloron had limited the record of French claims to the burial of lead plates, Marin constructed and garrisoned forts. He first constructed Fort Presque Isle (near present-day Erie, Pennsylvania) on Lake Erie's south shore. He had a road built to the headwaters of LeBoeuf Creek. Marin constructed a second fort at Fort Le Boeuf (present-day Waterford, Pennsylvania), designed to guard the headwaters of LeBoeuf Creek. As he moved south, he drove off or captured British traders, alarming both the British and the Iroquois. Tanaghrisson, a chief of the Mingo, who were remnants of Iroquois and other tribes who had been driven west 

In [ ]:
tiny_datasets['triviaqa'][0]['answer']['aliases']

['Tap Dance',
 'Tapdance',
 'Soft shoe dance',
 'Tap danced',
 'Softshoe',
 'Tap shoe',
 'Hoofers',
 'Tap-dance',
 'Tapdancing',
 'Tap dancers',
 'Tap dancer',
 'Tap-dancing',
 'Tap dancing',
 'Hoofer',
 'Tap Dancing',
 'Heel clicking',
 'Soft shoe',
 'Tap (dance)',
 'Tap dance']

### Evalute Basic Experiment

In [ ]:
def evaluate_basic_experiment(sampled_datasets={}, generated_answers={}):
  """
    Run a basic experiment to generate answers and evaluate them across datasets.

    Args:
        helper (Llama7BHelper): An instance of the Llama7BHelper class.
        sampled_datasets (dict): A dictionary of sampled datasets.

    Returns:
        dict: A dictionary containing evaluation results for each dataset.
    """

  # Initialize a dictionary to store evaluation results
  evaluation_results = {}

  for dataset_name, answers in generated_answers.items():
      print(f"\nEvaluating answers for {dataset_name}...")
      if dataset_name == 'squad':
        references = [sample["answers"]['text'] for sample in sampled_datasets["squad"]]
      elif dataset_name == 'hotpotqa':
        references = [[sample["answer"]] for sample in sampled_datasets["hotpotqa"]]
      elif dataset_name == 'triviaqa':
        references = [sample["answer"]['aliases'] for sample in sampled_datasets["triviaqa"]]
            #what is the color of the sky
            #I say: blue
            #triviqa: "blue, BLUE, blUE, bleu, cyan, aqua", etc.
      else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

      # Evaluate answers
      metrics = evaluate_answers(answers, references)
      # Interpret metrics
      # interpreted_metrics = interpret_metrics(metrics)
      evaluation_results[dataset_name] = metrics

      # Display interpreted metrics
      # for metric, details in interpreted_metrics.items():
      #     print(f"{metric.upper()}: {details['score']} - {details['description']}")
  return evaluation_results

In [ ]:
# Select the first 5 rows of each dataset
tiny_datasets = {
    dataset_name: dataset.select(range(10))  # Select the first 5 rows
    for dataset_name, dataset in smaller_datasets.items()
}

# Verify the result
for name, data in tiny_datasets.items():
    print(f"Dataset: {name}, Number of Examples: {data.num_rows}")

Dataset: squad, Number of Examples: 10
Dataset: hotpotqa, Number of Examples: 10
Dataset: triviaqa, Number of Examples: 10


In [ ]:
print(tiny_datasets["hotpotqa"][:1])


{'id': ['5add1d575542992c1e3a2540'], 'question': ["What nationality was Oliver Reed's character in the film Royal Flash?"], 'answer': ['Prussian'], 'type': ['bridge'], 'level': ['hard'], 'supporting_facts': [{'sent_id': [0, 2, 0], 'title': ['Royal Flash (film)', 'Royal Flash (film)', 'Otto von Bismarck']}], 'context': [{'sentences': [["Robin Barton (born 5 November 1958) is a British art dealer dealing primarily with Banksy's.", ' Barton studied photography and graphic design at the Exeter College of Art and Design and this was his first encounter with Russell Young.', ' Moving to London in 1980 he began working as a freelance photographer for music and fashion publications "Sounds", "NME", "Blitz", "The Face" moving on to working regularly for pioneering "Independent Magazine" photographing amongst others Sir Alec Guinness, Oliver Reed, Johnny Depp, Lou Reed, Hugh Grant and Sir Peter Hall.', ' Laterly he worked for other publications "Sunday Times", "Sunday Telegraph", "Elle", "Vogue"

### Execute Experiment & Evaluate Results

In [ ]:
# # print(tiny_datasets["triviaqa"][:5])
# counter = 0
# for idx, sample in enumerate(tiny_datasets["triviaqa"]):
#   if counter <1:
#     print(f"Sample {idx + 1}:")
#     print(sample.keys())
#     print(f"Question: {sample['question']}")

#     answers = sample.get("answer", {})
#     print(answers)
#     print(sample["answer"].keys())
#     counter+=1


In [ ]:
generated_answers = run_basic_experiment(helper, tiny_datasets)


Generating answers for squad...


Generating Answers:   0%|          | 0/10 [00:00<?, ?it/s]


Generating answers for hotpotqa...


Generating Answers:   0%|          | 0/10 [00:00<?, ?it/s]


Generating answers for triviaqa...


Generating Answers:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# generated_answers.keys()

dict_keys(['squad', 'hotpotqa', 'triviaqa'])

In [ ]:
# generated_answers["squad"]
# for k in generated_answers["squad"].items():

#   print(f"This is the predicted answer: {k}")

['About the beginning of the eighteenth century, the Spanish and Indians established settlements in the area. It is believed that some of the earliest settlers were Indians from Sonoma who came from Sonoma to Point Reyes, a distance of more than 30 miles.\nQuestion',
 'When the net force is zero, there must be equal and opposite reaction forces on every body involved to make the vector',
 'India was the country Thoreau dreamed of visiting and he never got a chance to. So, it’s fair to say, Thoreau would write something that would describe the India that he dreamed of it being. He probably would be very critical of the way India exists now and would be angry at how industrialization (and globalization, particularly) has taken away the culture of India. But the India that he would have dreamed of and wrote about, would have to',
 'A Papal Bull\nQuestion: In 1809, Napoleon tried to impose a state of sie',
 'The U.S. government provides social services in the form of basic medical care and

In [ ]:
tiny_datasets['squad'][1]

{'id': '5ad26a5fd7d075001a42931b',
 'title': 'Force',
 'context': 'Pushing against an object on a frictional surface can result in a situation where the object does not move because the applied force is opposed by static friction, generated between the object and the table surface. For a situation with no movement, the static friction force exactly balances the applied force resulting in no acceleration. The static friction increases or decreases in response to the applied force up to an upper limit determined by the characteristics of the contact between the surface and the object.',
 'question': 'What increases or decreases in response to static friction?',
 'answers': {'answer_start': [], 'text': []}}

In [ ]:
evaluation_results = evaluate_basic_experiment(tiny_datasets, generated_answers)

# Access evaluation results for each dataset
for dataset_name, metrics in evaluation_results.items():
    print(f"\nFinal Metrics for {dataset_name}:")
    for metric, details in metrics.items():
        print(f"{metric.upper()}: {details['score']} - {details['description']}")


Evaluating answers for squad...
['Marin built a small fort with a palisade and a few huts to protect the shipbuilding shipyards.', "The magnitude of the static friction always increases when you decrease the shear stress. The magnitude of the static friction always decreases when you decrease the normal load .\nThe friction between two surfaces is called static friction. We need to determine the coefficient of static friction (μ) of two bodies. We can write the law of static friction with the help of Newton's second law .\nThe maximum limit of the normal force we can apply for a body is called the", '“I know of no more encouraging fact than the unquestioned ability of man to elevate his life by conscious endeavor.”\nThis quote encapsulates a great deal about the philosophy behind the idea of ‘Consciousness and Social Transformation’ program, that I am privilege to be a part of at the Satyagraha Institute in Pune, India.\nAfter a series of discussions and interactions, it has been deci

ValueError: attempt to get argmax of an empty sequence

In [ ]:
for dataset_name, metrics in evaluation_results.items():
  print(dataset_name, metrics)

squad {'rouge1': {'score': 0.0045, 'description': 'Low content coverage.'}, 'rouge2': {'score': 0.0, 'description': 'Low phrase overlap.'}, 'rougel': {'score': 0.0045, 'description': 'No interpretation available.'}, 'bleu': {'score': 0.0, 'description': 'No interpretation available.'}, 'bertscore_precision': {'score': 0.1606, 'description': 'No interpretation available.'}, 'bertscore_recall': {'score': 0.1302, 'description': 'No interpretation available.'}, 'bertscore_f1': {'score': 0.1438, 'description': 'Weak semantic alignment.'}, 'sts': {'score': 0.0012, 'description': 'Poor semantic coherence.'}}
hotpotqa {'rouge1': {'score': 0.0051, 'description': 'Low content coverage.'}, 'rouge2': {'score': 0.0, 'description': 'Low phrase overlap.'}, 'rougel': {'score': 0.0051, 'description': 'No interpretation available.'}, 'bleu': {'score': 0.0, 'description': 'No interpretation available.'}, 'bertscore_precision': {'score': 0.7868, 'description': 'No interpretation available.'}, 'bertscore_r

In [ ]:
plot_metrics(evaluation_results["squad"])

KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['rouge1', 'rouge2', 'rougeL', 'bleu', 'BERTScore_F1', 'STS']"

In [ ]:
save_evaluation_results(evaluation_results, save_directory)

### Run Greedy Decoding Experiment

In [ ]:
# Initialize a dictionary to store generated answers
generated_answers = {}

for dataset_name, dataset in sampled_datasets.items():
    print(f"\nGenerating answers for {dataset_name}...")
    if dataset_name == 'squad':
        questions = dataset['question']
    elif dataset_name == 'hotpotqa':
        questions = dataset['question']
    elif dataset_name == 'triviaqa':
        questions = [item['query'] for item in dataset['question']]
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    answers = generate_answers(model, tokenizer, questions, do_sample=False) #setting to False for greedy
    generated_answers[dataset_name] = answers


# Initialize a dictionary to store evaluation results
evaluation_results = {}

for dataset_name, answers in generated_answers.items():
    print(f"\nEvaluating answers for {dataset_name}...")
    if dataset_name == 'squad':
        references = sampled_datasets['squad']['answers']['text']
    elif dataset_name == 'hotpotqa':
        references = sampled_datasets['hotpotqa']['answer']
    elif dataset_name == 'triviaqa':
        references = [item['answer']['value'][0] if isinstance(item['answer']['value'], list) else item['answer']['value'] for item in sampled_datasets['triviaqa']]
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    # Evaluate answers
    metrics = evaluate_answers(answers, references)
    # Interpret metrics
    interpreted_metrics = interpret_metrics(metrics)
    evaluation_results[dataset_name] = interpreted_metrics

    # Display interpreted metrics
    for metric, details in interpreted.items():
        print(f"{metric.upper()}: {details['score']} - {details['description']}")

In [ ]:
plot_evaluation_metrics(evaluation_results)

In [ ]:
save_evaluation_results(evaluation_results, save_directory)

## Run Beam Search Experiment



In [ ]:
# Process each dataset and evaluate with beam search
results = {}
beam_width = 5  # Adjust based on your computational resources

for name in dataset_names:
    if name in datasets_loaded:
        print(f"\nStarting evaluation for {name} with beam width = {beam_width}...")
        metrics = evaluate_manual_beam_search(
            helper=helper,
            dataset=datasets_loaded[name],
            max_length=100,
            num_beams=beam_width
        )
        results[name] = metrics
save_evaluation_results(results, save_directory, filename='llama2_7b_qa_semantic_coherence_metrics.csv')

plot_metrics(results)

## Running experiment

In [ ]:
# Example usage:
helper = Llama7BHelper(pretrained_model="huggyllama/llama-7b")
prompt = "Explain the theory of relativity in simple terms."
generated_text = manual_beam_search(
    helper=helper,
    prompt=prompt,
    max_length=50,
    num_beams=5,
    early_stopping=True,
    no_repeat_ngram_size=2
)
print("Generated Text:", generated_text)

# TODO:

In [ ]:
'''
NEED TO CALL FUNCTION TO GENERATE STILL
generate baseline performance

evaluate semantic coherence

generate beam search baseline

activation baseline


# Interpret the metrics dynamically
interpreted_metrics_dynamic = interpret_metrics_dynamic(evaluation_scores)

# Display the results
for metric, details in interpreted_metrics_dynamic.items():
    print(f"{metric.upper()}: {details['score']} - {details['description']}")

Nina does: https://github.com/nrimsky/LM-exp/blob/main/steering/activation_engineering.ipynb
return self.tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
--> should I be doing batch_decode?


Using up GPU memory
https://discuss.pytorch.org/t/how-can-we-release-gpu-memory-cache/14530
try clearing it

'''